In [2]:
source("helper.R")

max_expid<-get_max("Log")+1
exp_name<-"sensitivityOfStartPoint"
log_dir<-glue("Log/{max_expid}.{exp_name}")
logger=get_logger(log_dir, log_name = "log", debug = TRUE)
futile.logger::flog.info("start experiment", name = "log")

## get logger

use_saudi<-FALSE
seed_random<-123
n=30

futile.logger::flog.info(glue("use_saudi: {use_saudi}"), name = "log")

set.seed(seed_random)

futile.logger::flog.info(glue("use seed: {seed_random}"), name = "log")

futile.logger::flog.info(glue("number of experiments: {n}"), name = "log")

if(!use_saudi){

    data_ori <- as.data.frame(read_csv("data/IFclose.csv"))
    logpt<-log(data_ori$close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)]
    date <- as.Date(as.character(data_ori$Date[1:(length(data_ori$Date)-1)]), format = "%Y%m%d")
    date_timeindex<-as.numeric(as.POSIXct(date)) 
    date_timeindex <- (date_timeindex - min(date_timeindex))/86400 ## 1 day = 86400 seconds
    mesh <- fm_mesh_1d(date_timeindex)
    data = list(logreturn = logreturn, times = date_timeindex)
}else{
    data_ori <- as.data.frame(read_csv("data/1180_vol.csv"))
    logpt<-log(data_ori$Close)
    logreturn <-logpt[2:length(logpt)]-logpt[1:(length(logpt)-1)] ## 1 day = 86400 seconds
    date <- data_ori$Date[1:(length(data_ori$Date)-1)]
    date_timeindex<-as.numeric(as.POSIXct(date)) 
    date_timeindex <- (date_timeindex - min(date_timeindex))/86400
    mesh <- fm_mesh_1d(date_timeindex)
    data = list(logreturn = logreturn, times = date_timeindex)
}



INFO [2025-08-06 19:03:08] start experiment
INFO [2025-08-06 19:03:08] use_saudi: FALSE


INFO [2025-08-06 19:03:08] use seed: 123
INFO [2025-08-06 19:03:08] number of experiments: 30


Rows: 1595 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): ins_id
dbl (2): Date, close

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [ ]:


params <- data.frame(
  rspde.order = sample(2:6, n, replace = TRUE),
  prior.range.nominal = runif(n, 5, 300),
  prior.std.dev.nominal = runif(n, 0.3, 1.7)
)




# 

for(model_family in c("stochvol","stochvol.t","stochvol.nig")){
  results_list <- list()
  mlik_vec <- numeric(n)


  futile.logger::flog.info(glue("Start fitting models:{model_family}"), name = "log")
  for (i in 1:n) {
    futile.logger::flog.info(glue("🔁 Fitting model {i} of {n} ..."), name = "log")
    
    # 提取参数
    rspde.order <- params$rspde.order[i]
    prior.range.nominal <- params$prior.range.nominal[i]
    prior.std.dev.nominal <- params$prior.std.dev.nominal[i]
    
    # 构建模型
    OU_model <- rspde.matern(
      mesh = mesh,
      parameterization = "matern",
      rspde.order = rspde.order,
      prior.range.nominal = prior.range.nominal,
      prior.std.dev.nominal = prior.std.dev.nominal
    )
    
    formula_OU <- logreturn ~ field(times, model = OU_model)
    
    fit <- tryCatch(
      bru(
        formula_OU,
        family = "stochvol.t",
        data = data
      ),
      error = function(e) {
        futile.logger::flog.warn(glue("⚠️ Fit failed at i = {i} : {e$message}"), name = "log")
        return(NULL)
      }
    )
    
    # 如果成功拟合
    if (!is.null(fit)) {
      result_fit <- rspde.result(fit, "field", OU_model, parameterization = "matern")
      mlik_vec[i] <- fit$mlik[2]  # marginal log-likelihood
      results_list[[i]] <- list(
        param = params[i, ],
        mlik = fit$mlik[2],
        result = result_fit
      )
    } else {
      mlik_vec[i] <- NA
      results_list[[i]] <- list(param = params[i, ], mlik = NA, result = NULL)
    }
  }
  sorted_idx <- order(mlik_vec, decreasing = TRUE)


  results_list_sorted <- results_list[sorted_idx]


  table_out <- data.frame(
    param = character(),
    std.dev = numeric(),
    range = numeric(),
    nu = numeric(),
    likelihood = numeric(),
    stringsAsFactors = FALSE
  )
  results_list_sorted <- results_list[sorted_idx]
  for (i in seq_along(results_list_sorted)) {
    res <- results_list_sorted[[i]]
    
    if (is.null(res$result)) next 
    
    param_str <- with(res$param, glue::glue("({rspde.order}/{round(prior.range.nominal, 4)}/{round(prior.std.dev.nominal, 4)})"))
    
    summary_res <- summary(res$result)
    
    table_out[i, ] <- list(
      param = param_str,
      std.dev = round(summary_res$mean[1], 3),
      range   = round(summary_res$mean[2], 3),
      nu      = round(summary_res$mean[3], 3),
      likelihood = round(res$mlik, 3)
    )
  }

  table_out <- na.omit(table_out)


  table_string<-knitr::kable(
    table_out,
    format = "latex",
    booktabs = TRUE,
    linesep = ""
  )

  file_name <- glue("{log_dir}/model_with_{model_family}.tex")
  writeLines(table_string,  file_name)

}

INFO [2025-08-06 19:03:08] Start fitting models:stochvol
INFO [2025-08-06 19:03:08] 🔁 Fitting model 1 of 30 ...


Warning message in bru_log_warn(paste0("Non data-frame list-like data supplied; ", :
“Non data-frame list-like data supplied; guessing allow_combine=TRUE.
  Specify allow_combine explicitly to avoid this warning.”
Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-06 19:03:18] 🔁 Fitting model 2 of 30 ...


Warning message in bru_log_warn(paste0("Non data-frame list-like data supplied; ", :
“Non data-frame list-like data supplied; guessing allow_combine=TRUE.
  Specify allow_combine explicitly to avoid this warning.”
Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-06 19:03:27] 🔁 Fitting model 3 of 30 ...


...

Warning message in bru_log_warn(paste0("Non data-frame list-like data supplied; ", :
“Non data-frame list-like data supplied; guessing allow_combine=TRUE.
  Specify allow_combine explicitly to avoid this warning.”
Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-06 19:08:01] Start fitting models:stochvol.t
INFO [2025-08-06 19:08:01] 🔁 Fitting model 1 of 30 ...


Warning message in bru_log_warn(paste0("Non data-frame list-like data supplied; ", :
“Non data-frame list-like data supplied; guessing allow_combine=TRUE.
  Specify allow_combine explicitly to avoid this warning.”
Warning message in rspde.result(fit, "field", OU_model, parameterization = "matern"):
“the mean or mode of nu is very close to nu.upper.bound, please consider increasing nu.upper.bound, and refitting the model.”


INFO [2025-08-06 19:08:11] 🔁 Fitting model 2 of 30 ...


Warning message in bru_log_warn(paste0("Non data-frame list-like data supplied; ", :
“Non data-frame list-like data supplied; guessing allow_combine=TRUE.
  Specify allow_combine explicitly to avoid this warning.”
